<a href="https://colab.research.google.com/github/danielhladky/colab_fpp/blob/master/pandas_dataframe_read_excel_from_local_file_part1_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Learn Pandas based on the tutorial on***
http://pbpython.com/excel-pandas-comp.html
http://pbpython.com/excel-pandas-comp-2.html

In [2]:
# import data to colab based on https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
import pandas as pd
import numpy as np
# url = "https://github.com/danielhladky/datasets/blob/master/excel-comp-data.xlsx"

from google.colab import files
uploaded = files.upload()

Saving excel-comp-data.xlsx to excel-comp-data.xlsx


In [3]:
# store the file into a dataframe
import io
df = pd.read_excel(io.BytesIO(uploaded['excel-comp-data.xlsx']))
df.head()

,account,name,street,city,state,postal-code,Jan,Feb,Mar
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752,10000,62000,35000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365,95000,45000,35000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517,91000,120000,35000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021,45000,120000,10000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681,162000,120000,35000


Create a toal column in order sum Jan,Feb,Mar in XLS =sum(G2:I2)

In [4]:
df["total"] = df['Jan'] + df["Feb"] + df["Mar"]
df.head()

,account,name,street,city,state,postal-code,Jan,Feb,Mar,total
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752,10000,62000,35000,107000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365,95000,45000,35000,175000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517,91000,120000,35000,246000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021,45000,120000,10000,175000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681,162000,120000,35000,317000


Perfome some analysis by column Jan, Feb and Mar. Like a new row at the end of each column that shows some analytics. E.g. sum, min, max, mean

In [5]:
# sum over Jan, mean, min, max
df["Jan"].sum(), df["Jan"].mean(), df["Jan"].min(), df["Jan"].max()

(1462000, 97466.66666666667, 10000, 162000)

Add the sum as a new row to the dataframe

In [6]:
# first, create sum for each column you want
sum_row=df[["Jan", "Feb", "Mar", "total"]].sum()
sum_row

Jan      1462000
Feb      1507000
Mar       717000
total    3686000
dtype: int64

In [7]:
# second, transpose into a dataframe
df_sum=pd.DataFrame(data=sum_row).T
df_sum

,Jan,Feb,Mar,total
0,1462000,1507000,717000,3686000


In [8]:
# thirs, reindex the df
df_sum=df_sum.reindex(columns=df.columns)
df_sum

,account,name,street,city,state,postal-code,Jan,Feb,Mar,total
0,NaN,NaN,NaN,NaN,NaN,NaN,1462000,1507000,717000,3686000


In [10]:
# replace NaN in state with ' '
df_sum.state = (' ')
df_sum

,account,name,street,city,state,postal-code,Jan,Feb,Mar,total
0,NaN,NaN,NaN,NaN,,NaN,1462000,1507000,717000,3686000


In [11]:
# add the new dataframe, the new row to the existing dataframe. create a new df_final that combines all
df_final = df.append(df_sum, ignore_index=True)
# show the last records
df_final.tail()

,account,name,street,city,state,postal-code,Jan,Feb,Mar,total
11,231907.0,Hahn-Moore,18115 Olivine Throughway,Norbertomouth,NorthDakota,31415.0,150000,10000,162000,322000
12,242368.0,"Frami, Anderson and Donnelly",182 Bertie Road,East Davian,Iowa,72686.0,162000,120000,35000,317000
13,268755.0,Walsh-Haley,2624 Beatty Parkways,Goodwinmouth,RhodeIsland,31919.0,55000,120000,35000,210000
14,273274.0,McDermott PLC,8917 Bergstrom Meadow,Kathryneborough,Delaware,27933.0,150000,120000,70000,340000
15,NaN,NaN,NaN,NaN,,NaN,1462000,1507000,717000,3686000


Add a new colum with the abbreviations of the state, e.g. Vermont = VT. In excel you can use vlookup but problem will arise when spüelling is not correct. Pndas can support this with the library fuzzy wuzzy.
In case a python library is not in colab install it with pip

In [12]:
!pip install fuzzywuzzy

In [13]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [0]:
# convert all state to code
state_to_code = {"VERMONT": "VT", "GEORGIA": "GA", "IOWA": "IA", "Armed Forces Pacific": "AP", "GUAM": "GU",
                 "KANSAS": "KS", "FLORIDA": "FL", "AMERICAN SAMOA": "AS", "NORTH CAROLINA": "NC", "HAWAII": "HI",
                 "NEW YORK": "NY", "CALIFORNIA": "CA", "ALABAMA": "AL", "IDAHO": "ID", "FEDERATED STATES OF MICRONESIA": "FM",
                 "Armed Forces Americas": "AA", "DELAWARE": "DE", "ALASKA": "AK", "ILLINOIS": "IL",
                 "Armed Forces Africa": "AE", "SOUTH DAKOTA": "SD", "CONNECTICUT": "CT", "MONTANA": "MT", "MASSACHUSETTS": "MA",
                 "PUERTO RICO": "PR", "Armed Forces Canada": "AE", "NEW HAMPSHIRE": "NH", "MARYLAND": "MD", "NEW MEXICO": "NM",
                 "MISSISSIPPI": "MS", "TENNESSEE": "TN", "PALAU": "PW", "COLORADO": "CO", "Armed Forces Middle East": "AE",
                 "NEW JERSEY": "NJ", "UTAH": "UT", "MICHIGAN": "MI", "WEST VIRGINIA": "WV", "WASHINGTON": "WA",
                 "MINNESOTA": "MN", "OREGON": "OR", "VIRGINIA": "VA", "VIRGIN ISLANDS": "VI", "MARSHALL ISLANDS": "MH",
                 "WYOMING": "WY", "OHIO": "OH", "SOUTH CAROLINA": "SC", "INDIANA": "IN", "NEVADA": "NV", "LOUISIANA": "LA",
                 "NORTHERN MARIANA ISLANDS": "MP", "NEBRASKA": "NE", "ARIZONA": "AZ", "WISCONSIN": "WI", "NORTH DAKOTA": "ND",
                 "Armed Forces Europe": "AE", "PENNSYLVANIA": "PA", "OKLAHOMA": "OK", "KENTUCKY": "KY", "RHODE ISLAND": "RI",
                 "DISTRICT OF COLUMBIA": "DC", "ARKANSAS": "AR", "MISSOURI": "MO", "TEXAS": "TX", "MAINE": "ME"}

In [15]:
# check how fuzzywuzzy works
process.extractOne("Minnesotta", choices=state_to_code.keys())

('MINNESOTA', 95)

In [16]:
# use the score_cutoff to specify how precise fuzzy should work
process.extractOne("AlaBAMMazzz",choices=state_to_code.keys(),score_cutoff=60)

('ALABAMA', 78)

Apply the fuzzywuzzy as a function to convert the strings into abbreviations. The functions returns only values if found otherwise np.nan (numpy nan) 

In [0]:
def convert_state(row):
  abbrev = process.extractOne(row["state"],choices=state_to_code.keys(),score_cutoff=80)
  if abbrev:
    return state_to_code[abbrev[0]]
  return np.nan

In [18]:
# initialise the df with a new column and fill with np.nan
df_final.insert(6, "abbrev", np.nan)
df_final.head()

,account,name,street,city,state,postal-code,abbrev,Jan,Feb,Mar,total
0,211829.0,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,Texas,28752.0,NaN,10000,62000,35000,107000
1,320563.0,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NorthCarolina,38365.0,NaN,95000,45000,35000,175000
2,648336.0,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,Iowa,76517.0,NaN,91000,120000,35000,246000
3,109996.0,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,Maine,46021.0,NaN,45000,120000,10000,175000
4,121213.0,Bauch-Goldner,7274 Marissa Common,Shanahanchester,California,49681.0,NaN,162000,120000,35000,317000


In [21]:
# fill abbrev according state using the function
df_final['abbrev'] = df_final.apply(convert_state, axis=1)
df_final.tail()

,account,name,street,city,state,postal-code,abbrev,Jan,Feb,Mar,total
11,231907.0,Hahn-Moore,18115 Olivine Throughway,Norbertomouth,NorthDakota,31415.0,ND,150000,10000,162000,322000
12,242368.0,"Frami, Anderson and Donnelly",182 Bertie Road,East Davian,Iowa,72686.0,IA,162000,120000,35000,317000
13,268755.0,Walsh-Haley,2624 Beatty Parkways,Goodwinmouth,RhodeIsland,31919.0,RI,55000,120000,35000,210000
14,273274.0,McDermott PLC,8917 Bergstrom Meadow,Kathryneborough,Delaware,27933.0,DE,150000,120000,70000,340000
15,NaN,NaN,NaN,NaN,,NaN,NaN,1462000,1507000,717000,3686000


In [22]:
# see the complete dataframe via print
print(df_final.to_string())

     account                              name                                street               city          state  postal-code abbrev      Jan      Feb     Mar    total
0   211829.0        Kerluke, Koepp and Hilpert                    34456 Sean Highway         New Jaycob          Texas      28752.0     TX    10000    62000   35000   107000
1   320563.0                    Walter-Trantow                     1311 Alvis Tunnel      Port Khadijah  NorthCarolina      38365.0     NC    95000    45000   35000   175000
2   648336.0        Bashirian, Kunde and Price  62184 Schamberger Underpass Apt. 231     New Lilianland           Iowa      76517.0     IA    91000   120000   35000   246000
3   109996.0       D'Amore, Gleichner and Bode           155 Fadel Crescent Apt. 144         Hyattburgh          Maine      46021.0     ME    45000   120000   10000   175000
4   121213.0                     Bauch-Goldner                   7274 Marissa Common    Shanahanchester     California      49681.

Adding subtotals to the table using the groupby function in pandas

In [23]:
df_sub = df_final[["abbrev", "Jan", "Feb", "Mar", "total"]].groupby('abbrev').sum()
df_sub

,Jan,Feb,Mar,total
abbrev,,,,
AR,150000,120000,35000,305000
CA,162000,120000,35000,317000
DE,150000,120000,70000,340000
IA,253000,240000,70000,563000
ID,70000,120000,35000,225000
ME,45000,120000,10000,175000
MS,62000,120000,70000,252000
NC,95000,45000,35000,175000
ND,150000,10000,162000,322000


In [25]:
# create a function to format column with specific format values
def money(x):
  return "${:,.0f}".format(x)

formatted_df = df_sub.applymap(money)
formatted_df

,Jan,Feb,Mar,total
abbrev,,,,
AR,"$150,000","$120,000","$35,000","$305,000"
CA,"$162,000","$120,000","$35,000","$317,000"
DE,"$150,000","$120,000","$70,000","$340,000"
IA,"$253,000","$240,000","$70,000","$563,000"
ID,"$70,000","$120,000","$35,000","$225,000"
ME,"$45,000","$120,000","$10,000","$175,000"
MS,"$62,000","$120,000","$70,000","$252,000"
NC,"$95,000","$45,000","$35,000","$175,000"
ND,"$150,000","$10,000","$162,000","$322,000"


In [26]:
# create totals for the df_sub and convert to a new df format and append
sum_row=df_sub[["Jan","Feb","Mar","total"]].sum()
sum_row

Jan      1462000
Feb      1507000
Mar       717000
total    3686000
dtype: int64

In [27]:
df_sub_sum=pd.DataFrame(data=sum_row).T
df_sub_sum=df_sub_sum.applymap(money)
df_sub_sum

,Jan,Feb,Mar,total
0,"$1,462,000","$1,507,000","$717,000","$3,686,000"


In [28]:
final_table = formatted_df.append(df_sub_sum)
final_table

,Jan,Feb,Mar,total
AR,"$150,000","$120,000","$35,000","$305,000"
CA,"$162,000","$120,000","$35,000","$317,000"
DE,"$150,000","$120,000","$70,000","$340,000"
IA,"$253,000","$240,000","$70,000","$563,000"
ID,"$70,000","$120,000","$35,000","$225,000"
ME,"$45,000","$120,000","$10,000","$175,000"
MS,"$62,000","$120,000","$70,000","$252,000"
NC,"$95,000","$45,000","$35,000","$175,000"
ND,"$150,000","$10,000","$162,000","$322,000"
PA,"$70,000","$95,000","$35,000","$200,000"


In [29]:
# rename in the index the 0 with total
final_table = final_table.rename(index={0:"Total"})
final_table

,Jan,Feb,Mar,total
AR,"$150,000","$120,000","$35,000","$305,000"
CA,"$162,000","$120,000","$35,000","$317,000"
DE,"$150,000","$120,000","$70,000","$340,000"
IA,"$253,000","$240,000","$70,000","$563,000"
ID,"$70,000","$120,000","$35,000","$225,000"
ME,"$45,000","$120,000","$10,000","$175,000"
MS,"$62,000","$120,000","$70,000","$252,000"
NC,"$95,000","$45,000","$35,000","$175,000"
ND,"$150,000","$10,000","$162,000","$322,000"
PA,"$70,000","$95,000","$35,000","$200,000"


Common Excel Tasks Demonstrated in Pandas - Part 2
http://pbpython.com/excel-pandas-comp-2.html
File: sample-salesv3.xlsx
Colab: Python3

In [4]:
import pandas as pd
import numpy as np
# read new xls file "sample-salesv3.xlsx"
from google.colab import files
uploaded = files.upload()

Saving sample-salesv3.xlsx to sample-salesv3.xlsx


In [5]:
# store the file into a dataframe
import io
df = pd.read_excel(io.BytesIO(uploaded['sample-salesv3.xlsx']))
df.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2014-01-01 23:26:55


In [32]:
# check the different datatypes
df.dtypes

account number      int64
name               object
sku                object
quantity            int64
unit price        float64
ext price         float64
date               object
dtype: object

In [6]:
# convert data which is type object into datetime
df['date'] = pd.to_datetime(df['date'])
df.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2014-01-01 23:26:55


In [34]:
df.dtypes

account number             int64
name                      object
sku                       object
quantity                   int64
unit price               float64
ext price                float64
date              datetime64[ns]
dtype: object

In [35]:
# filter the dataframe by a specific colum and value
df[df["account number"]==307599].head()

,account number,name,sku,quantity,unit price,ext price,date
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
13,307599,"Kassulke, Ondricka and Metz",S2-10342,17,12.44,211.48,2014-01-04 07:53:01
34,307599,"Kassulke, Ondricka and Metz",S2-78676,35,33.04,1156.40,2014-01-10 05:26:31
58,307599,"Kassulke, Ondricka and Metz",B1-20000,22,37.87,833.14,2014-01-15 16:22:22
70,307599,"Kassulke, Ondricka and Metz",S2-10342,44,96.79,4258.76,2014-01-18 06:32:31


In [36]:
# filter a column with numberic values
df[df["quantity"] > 22].head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
14,737550,"Fritsch, Russel and Anderson",B1-53102,23,71.56,1645.88,2014-01-04 08:57:48
15,239344,Stokes LLC,S1-06532,34,71.51,2431.34,2014-01-04 11:34:58


In [37]:
# using map and lambda provides a reach filterign and mapping
df[df["sku"].map(lambda x: x.startswith('B1'))].head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
6,218895,Kulas Inc,B1-65551,2,31.10,62.20,2014-01-02 10:57:23
14,737550,"Fritsch, Russel and Anderson",B1-53102,23,71.56,1645.88,2014-01-04 08:57:48
17,239344,Stokes LLC,B1-50809,14,16.23,227.22,2014-01-04 22:14:32


In [38]:
df[df["sku"].map(lambda x: x.startswith('B1')) & (df["quantity"] > 22)].head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
14,737550,"Fritsch, Russel and Anderson",B1-53102,23,71.56,1645.88,2014-01-04 08:57:48
26,737550,"Fritsch, Russel and Anderson",B1-53636,42,42.06,1766.52,2014-01-08 00:02:11
31,714466,Trantow-Barrows,B1-33087,32,19.56,625.92,2014-01-09 10:16:32


In [39]:
# use isin for a list of values
df[df["account number"].isin([714466,218895])].head()

,account number,name,sku,quantity,unit price,ext price,date
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
5,714466,Trantow-Barrows,S2-77896,17,87.63,1489.71,2014-01-02 10:07:15
6,218895,Kulas Inc,B1-65551,2,31.10,62.20,2014-01-02 10:57:23
8,714466,Trantow-Barrows,S1-50961,22,84.09,1849.98,2014-01-03 11:29:02


In [45]:
# run a query with filter and use time% to measure performance
%time
df.query('name == ["Kulas Inc","Barton LLC"]').head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
6,218895,Kulas Inc,B1-65551,2,31.10,62.20,2014-01-02 10:57:23
33,218895,Kulas Inc,S1-06532,3,22.36,67.08,2014-01-09 23:58:27
36,218895,Kulas Inc,S2-34077,16,73.04,1168.64,2014-01-10 12:07:30


In [46]:
# the same query with isin
%time
df[df["name"].isin(["Barton LLC","Kulas Inc"])].head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
6,218895,Kulas Inc,B1-65551,2,31.10,62.20,2014-01-02 10:57:23
33,218895,Kulas Inc,S1-06532,3,22.36,67.08,2014-01-09 23:58:27
36,218895,Kulas Inc,S2-34077,16,73.04,1168.64,2014-01-10 12:07:30


Working with Dates and Time in pandas

In [51]:
!pip show pandas

Name: pandas
Version: 0.22.0
Summary: Powerful data structures for data analysis, time series,and statistics
Home-page: http://pandas.pydata.org
Author: The PyData Development Team
Author-email: pydata@googlegroups.com
License: BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, pytz, python-dateutil
Required-by: xarray, vega-datasets, statsmodels, stable-baselines, pymc3, pandas-profiling, pandas-gbq, pandas-datareader, mlxtend, magenta, gspread-dataframe, fix-yahoo-finance, featuretools, fbprophet, cufflinks, altair


In [7]:
# sort pandas - note: version 0.22 https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.sort_values.html
df = df.sort_values(by=['date'])
df.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2014-01-01 23:26:55


In [53]:
# filter by date
df[df['date'] >='20140905'].head()

,account number,name,sku,quantity,unit price,ext price,date
1042,163416,Purdy-Kunde,B1-38851,41,98.69,4046.29,2014-09-05 01:52:32
1043,714466,Trantow-Barrows,S1-30248,1,37.16,37.16,2014-09-05 06:17:19
1044,729833,Koepp Ltd,S1-65481,48,16.04,769.92,2014-09-05 08:54:41
1045,729833,Koepp Ltd,S2-11481,6,26.50,159.00,2014-09-05 16:33:15
1046,737550,"Fritsch, Russel and Anderson",B1-33364,4,76.44,305.76,2014-09-06 08:59:08


In [54]:
# partial filter on date
df[df['date'] >='2014-03'].head()

,account number,name,sku,quantity,unit price,ext price,date
242,163416,Purdy-Kunde,S1-30248,19,65.03,1235.57,2014-03-01 16:07:40
243,527099,Sanford and Sons,S2-82423,3,76.21,228.63,2014-03-01 17:18:01
244,527099,Sanford and Sons,B1-50809,8,70.78,566.24,2014-03-01 18:53:09
245,737550,"Fritsch, Russel and Anderson",B1-50809,20,50.11,1002.20,2014-03-01 23:47:17
246,688981,Keeling LLC,B1-86481,-1,97.16,-97.16,2014-03-02 01:46:44


In [55]:
# chain filter criteria
df[(df['date'] >='20140701') & (df['date'] <= '20140715')].head()

,account number,name,sku,quantity,unit price,ext price,date
778,737550,"Fritsch, Russel and Anderson",S1-65481,35,70.51,2467.85,2014-07-01 00:21:58
779,218895,Kulas Inc,S1-30248,9,16.56,149.04,2014-07-01 00:52:38
780,163416,Purdy-Kunde,S2-82423,44,68.27,3003.88,2014-07-01 08:15:52
781,672390,Kuhn-Gusikowski,B1-04202,48,99.39,4770.72,2014-07-01 11:12:13
782,642753,Pollich LLC,S2-23246,1,51.29,51.29,2014-07-02 04:02:39


In [56]:
# because the date is in format datetime you can use also following
df[df['date'] >= 'Oct-2014'].head()

,account number,name,sku,quantity,unit price,ext price,date
1141,307599,"Kassulke, Ondricka and Metz",B1-50809,25,56.63,1415.75,2014-10-01 10:56:32
1142,737550,"Fritsch, Russel and Anderson",S2-82423,38,45.17,1716.46,2014-10-01 16:17:24
1143,737550,"Fritsch, Russel and Anderson",S1-47412,6,68.68,412.08,2014-10-01 22:28:49
1144,146832,Kiehn-Spinka,S2-11481,13,18.80,244.40,2014-10-02 00:31:01
1145,424914,White-Trantow,B1-53102,9,94.47,850.23,2014-10-02 02:48:26


In [8]:
# reindex the dataframe according date column
df2 = df.set_index(['date'])
df2.head()

,account number,name,sku,quantity,unit price,ext price
date,,,,,,
2014-01-01 07:21:51,740150,Barton LLC,B1-20000,39,86.69,3380.91
2014-01-01 10:00:47,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16
2014-01-01 13:24:58,218895,Kulas Inc,B1-69924,23,90.70,2086.10
2014-01-01 15:05:22,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05
2014-01-01 23:26:55,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26


In [9]:
# slide the data by date
df2["20140101":"20140201"].head()
# is the same as
# df2["2014-01-01":"2014-02-01"].head()

,account number,name,sku,quantity,unit price,ext price
date,,,,,,
2014-01-01 07:21:51,740150,Barton LLC,B1-20000,39,86.69,3380.91
2014-01-01 10:00:47,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16
2014-01-01 13:24:58,218895,Kulas Inc,B1-69924,23,90.70,2086.10
2014-01-01 15:05:22,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05
2014-01-01 23:26:55,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26


Use additional string functions to select, filater, map the data inside the dataframe

In [10]:
df[df['sku'].str.contains('B1')].head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
6,218895,Kulas Inc,B1-65551,2,31.10,62.20,2014-01-02 10:57:23
14,737550,"Fritsch, Russel and Anderson",B1-53102,23,71.56,1645.88,2014-01-04 08:57:48
17,239344,Stokes LLC,B1-50809,14,16.23,227.22,2014-01-04 22:14:32


In [11]:
# find unique items with dataframe
df["name"].unique()

array(['Barton LLC', 'Trantow-Barrows', 'Kulas Inc',
       'Kassulke, Ondricka and Metz', 'Jerde-Hilpert', 'Koepp Ltd',
       'Fritsch, Russel and Anderson', 'Kiehn-Spinka', 'Keeling LLC',
       'Frami, Hills and Schmidt', 'Stokes LLC', 'Kuhn-Gusikowski',
       'Herman LLC', 'White-Trantow', 'Sanford and Sons', 'Pollich LLC',
       'Will LLC', 'Cronin, Oberbrunner and Spencer',
       'Halvorson, Crona and Champlin', 'Purdy-Kunde'], dtype=object)

In [17]:
# check size of df - https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
#df.dtypes     
df.shape       # returns a tuple with number of rows and columns     

(1500, 7)

In [19]:
df.drop_duplicates(subset=["account number","name"]).head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2014-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2014-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2014-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2014-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2014-01-01 23:26:55


In [29]:
# drop duplicates and only show the first 3 columns "index, account number and name"
df.drop_duplicates(subset=["account number","name"]).iloc[:, 0:2]

,account number,name
0,740150,Barton LLC
1,714466,Trantow-Barrows
2,218895,Kulas Inc
3,307599,"Kassulke, Ondricka and Metz"
4,412290,Jerde-Hilpert
7,729833,Koepp Ltd
9,737550,"Fritsch, Russel and Anderson"
10,146832,Kiehn-Spinka
11,688981,Keeling LLC
12,786968,"Frami, Hills and Schmidt"
